<a href="https://colab.research.google.com/github/Parimala-15/student_dashboard/blob/main/student_dashboard_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install Dash Bootstrap Components
!pip install dash-bootstrap-components


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 51.6 MB/s eta 0:00:00


In [8]:
# -----------------------------
# Imports
# -----------------------------
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# -----------------------------
# Load CSV directly into Pandas
# -----------------------------
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Dashboard data.csv')

# -----------------------------
# Initialize Dash app
# -----------------------------
app = Dash(__name__)
app.title = "Student Dropout Dashboard"

# -----------------------------
# Layout with dropdowns, cards & headers
# -----------------------------
app.layout = html.Div([
    html.H1("📊 Student Dropout Analysis Dashboard",
            style={'textAlign':'center', 'marginBottom':'30px'}),

    # Dropdown filters
    html.Div([
        html.Div([
            html.Label("Select Module"),
            dcc.Dropdown(
                id='module-dropdown',
                options=[{'label': m, 'value': m} for m in sorted(df['code_module'].unique())],
                value=[sorted(df['code_module'].unique())[0]],
                multi=True
            ),
        ], style={'width':'48%', 'display':'inline-block', 'padding':'10px'}),

        html.Div([
            html.Label("Select Presentation"),
            dcc.Dropdown(
                id='presentation-dropdown',
                options=[{'label': p, 'value': p} for p in sorted(df['code_presentation'].unique())],
                value=[sorted(df['code_presentation'].unique())[0]],
                multi=True
            ),
        ], style={'width':'48%', 'display':'inline-block', 'padding':'10px'}),
    ], style={'marginBottom':'30px'}),

    # Graphs
    html.Div([
        html.Div([html.H3("Dropout Distribution"), dcc.Graph(id='dropout-bar')],
                 style={'width':'48%', 'display':'inline-block', 'verticalAlign':'top', 'padding':'10px'}),

        html.Div([html.H3("Engagement vs Dropout"), dcc.Graph(id='engagement-heatmap')],
                 style={'width':'48%', 'display':'inline-block', 'verticalAlign':'top', 'padding':'10px'}),
    ]),

    html.Div([
        html.Div([html.H3("Predicted Dropout Probabilities"), dcc.Graph(id='prediction-hist')],
                 style={'width':'48%', 'display':'inline-block', 'verticalAlign':'top', 'padding':'10px'}),

        html.Div([html.H3("Score vs Total Clicks"), dcc.Graph(id='score-scatter')],
                 style={'width':'48%', 'display':'inline-block', 'verticalAlign':'top', 'padding':'10px'}),
    ]),

    html.Div([
        html.Div([html.H3("Correlation Matrix"), dcc.Graph(id='corr-heatmap')],
                 style={'width':'48%', 'display':'inline-block', 'verticalAlign':'top', 'padding':'10px'}),

        html.Div([html.H3("Dropout Pie Chart"), dcc.Graph(id='dropout-pie')],
                 style={'width':'48%', 'display':'inline-block', 'verticalAlign':'top', 'padding':'10px'}),
    ]),

    # Overall metrics
    html.Div([
        html.H3("Overall Metrics", style={'textAlign':'center'}),
        html.Div([
            html.Div(id='total-students', style={'width':'33%', 'display':'inline-block', 'textAlign':'center'}),
            html.Div(id='dropout-percent', style={'width':'33%', 'display':'inline-block', 'textAlign':'center'}),
            html.Div(id='avg-clicks', style={'width':'33%', 'display':'inline-block', 'textAlign':'center'}),
        ])
    ], style={'marginTop':'30px'})
])

# -----------------------------
# Callback to update all graphs and metrics
# -----------------------------
@app.callback(
    Output('dropout-bar', 'figure'),
    Output('engagement-heatmap', 'figure'),
    Output('prediction-hist', 'figure'),
    Output('score-scatter', 'figure'),
    Output('corr-heatmap', 'figure'),
    Output('dropout-pie', 'figure'),
    Output('total-students', 'children'),
    Output('dropout-percent', 'children'),
    Output('avg-clicks', 'children'),
    Input('module-dropdown', 'value'),
    Input('presentation-dropdown', 'value')
)
def update_dashboard(selected_modules, selected_presentations):
    # Filter data based on dropdown selection
    filtered_df = df[df['code_module'].isin(selected_modules) & df['code_presentation'].isin(selected_presentations)]

    # Dropout distribution
    fig1 = px.histogram(filtered_df, x='dropout', color='dropout', barmode='group',
                        labels={'dropout':'Dropout'}, hover_data=filtered_df.columns)
    fig1.update_layout(title_text='Dropout Distribution', title_x=0.5, template='plotly_dark')

    # Engagement heatmap
    fig2 = px.density_heatmap(filtered_df, x='total_clicks', y='dropout', nbinsx=30, nbinsy=2,
                              color_continuous_scale='Viridis', labels={'total_clicks':'Total Clicks'})
    fig2.update_layout(title_text='Engagement vs Dropout', title_x=0.5, template='plotly_dark')

    # Predicted probabilities
    if 'prediction_prob' in filtered_df.columns:
        fig3 = px.histogram(filtered_df, x='prediction_prob', color='dropout', nbins=20,
                            labels={'prediction_prob':'Predicted Probability'}, hover_data=filtered_df.columns)
    else:
        fig3 = px.histogram(filtered_df, x='dropout', color='dropout', nbins=2)
    fig3.update_layout(title_text='Predicted Dropout Probabilities', title_x=0.5, template='plotly_dark')

    # Score vs Total Clicks scatter
    fig4 = px.scatter(filtered_df, x='total_clicks', y='score', color='dropout', hover_data=['id_student'],
                      labels={'total_clicks':'Total Clicks', 'score':'Score'})
    fig4.update_layout(title_text='Score vs Total Clicks', title_x=0.5, template='plotly_dark')

    # Correlation matrix (numeric only)
    numeric_df = filtered_df.select_dtypes(include=['int64','float64'])
    fig5 = px.imshow(numeric_df.corr(), text_auto=True, color_continuous_scale='Viridis')
    fig5.update_layout(title_text='Correlation Matrix', title_x=0.5, template='plotly_dark')

    # Dropout pie chart
    pie_counts = filtered_df['dropout'].value_counts()
    fig6 = px.pie(values=pie_counts.values, names=pie_counts.index, color=pie_counts.index)
    fig6.update_layout(title_text='Dropout Proportion', title_x=0.5, template='plotly_dark')

    # Metrics
    total_students = f"Total Students: {len(filtered_df)}"
    dropout_percent = f"Dropout %: {filtered_df['dropout'].mean()*100:.2f}%"
    avg_clicks = f"Average Clicks: {filtered_df['total_clicks'].mean():.2f}"

    return fig1, fig2, fig3, fig4, fig5, fig6, total_students, dropout_percent, avg_clicks

# -----------------------------
# Run app
# -----------------------------
if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=8050)


<IPython.core.display.Javascript object>